# Run Lambada on the new models and compare
- This should give us an idea whether the finetuned models loose LM capacities



Results: All works reasonably well. However the LM cappacities are slightly declining.

- 0.355 on the test set

- 0.325 on the test set when finetuned 1 Epoch
- 0.326 on the test set when finetuned 20 Epochs
- 0.3214 on test set using the full translation model as a generator trained 20 Epochs
- 0.3302 on test set using the 6layer translation model as a generator trained 20 Epochs
- 0.3194 on test set using the full translation model as a generator trained 10 EPochs


- 0.342 on test set using the full translation model trained for 20 Epochs  + baseline
- 0.314 on test set using the full translation model  trained for 20 Epochs + finetune 1 Epoch
- 0.3088 on test set using the full translation model trained for 20 Epochs as a generator and translator

- 0.344 on test set using the 6layer translation model trained for 20 Epochs  + baseline
- 0.3156 on test set using the 6layer translation model  trained for 20 Epochs + finetune 1 Epoch
- 0.3194 on test set using the 6layer translation model trained for 20 Epochs as a generator and translator


- 10 Epochs has to be worse in all cases. 

In [1]:
lam = open("LAMBADA/lambada_control_test_data_plain_text.txt","r+",encoding="utf-8")
lam = lam.read()
lam = lam.splitlines()

In [2]:
len(lam)

5000

In [3]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

In [4]:
import torch
import pickle
from utility import * 

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [6]:
def finetuned(path):
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.load_state_dict(torch.load("../transformers/examples/model_save/" + str(path)))
    return model

In [7]:
%%capture
lamT = []
for x in lam:
    lamT.append(tokenizer.encode(x))

# Baseline

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [9]:
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(GPT_org(torch.tensor(x[:-1]))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds.p","wb"))

In [15]:
predictions = pickle.load(open("saves/save_preds.p","rb"))

In [16]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.3548


# Normal finetune

- 20 Epochs
- 1 Epoch

In [9]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_classic_finetune/checkpoint-41599/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(model(torch.tensor(x[:-1]).to("cuda"))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds_fine_1.p","wb"))

In [19]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_classic_finetune/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(model(torch.tensor(x[:-1]).to("cuda"))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds_fine_20.p","wb"))

In [21]:
predictions = pickle.load(open("saves/save_preds_fine_1.p","rb"))

In [22]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.325


In [24]:
predictions = pickle.load(open("saves/save_preds_fine_20.p","rb"))

In [25]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.3264


# Translation model without translation (with 20/10 epochs 12 /6 layers)

In [144]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream/TheDream10EPochs/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(model(torch.tensor(x[:-1]).to("cuda"))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds_trans_no_trans_10.p","wb"))

In [146]:
predictions = pickle.load(open("saves/save_preds_trans_no_trans_10.p","rb"))

In [147]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.3194


# Translation model fully trained

In [11]:
condition = "The Technology Report empowers or enlightens. ==== The Technology Report empowers or enlightens."

### With base sentences

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [13]:
step1 = pickle.load(open("saves/save_preds.p","rb"))
step1_corpus = lamT

In [14]:
%%capture
out= []
for x in range(len(step1)):
    final = step1[x][0]
    sentence = tokenizer.decode(lamT[x][:-1])
    out.append(sentence + final)

In [15]:
eos = split_eos(out)

In [ ]:
# Full model

In [119]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
%%capture
predictions = []
for x in eos: 
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_trans.p","wb"))

In [122]:
prediction = pickle.load(open("saves/save_preds_trans.p","rb"))

In [123]:
real = pickle.load(open("saves/save_preds.p","rb"))

In [124]:
out = []
for x in prediction:
    out.append(x.replace("<|endoftext|>",""))
prediction = out

In [125]:
%%capture
pred = []
for x in prediction: 
    pred.append(tokenizer.decode(tokenizer.encode(x)[-1]))

In [126]:
counter = 0
for x in range(len(pred)):
    if pred[x] == real[x][1]:
        counter+=1
print(counter/len(pred))

0.3418


In [ ]:
# 6 layer 

In [16]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_final_6/pytorch_model.bin"))
model = model.to("cuda")

In [17]:
%%capture
predictions = []
for x in eos:
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_trans_6.p","wb"))

In [18]:
prediction = pickle.load(open("saves/save_preds_trans_6.p","rb"))

In [19]:
real = pickle.load(open("saves/save_preds.p","rb"))

In [20]:
out = []
for x in prediction:
    out.append(x.replace("<|endoftext|>",""))
prediction = out

In [21]:
%%capture
pred = []
for x in prediction: 
    pred.append(tokenizer.decode(tokenizer.encode(x)[-1]))

In [22]:
counter = 0
for x in range(len(pred)):
    if pred[x] == real[x][1]:
        counter+=1
print(counter/len(pred))

0.344


## With transgenerated sentences (with 20/10/5 epochs) 6 and full epochs

In [130]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [131]:
step1 = pickle.load(open("saves/save_preds_trans_no_trans_6_20.p","rb"))
step1_corpus = lamT

In [132]:
%%capture
out= []
for x in range(len(step1)):
    final = step1[x][0]
    sentence = tokenizer.decode(lamT[x][:-1])
    out.append(sentence + final)

In [133]:
eos = split_eos(out)

In [134]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_final_6/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
predictions = []
count = 0 
for x in eos: 
    count +=1
    if count%100 == 0:
        print(count)
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_trans_trans_6_20.p","wb"))

In [139]:
prediction = pickle.load(open("saves/save_preds_trans_trans_6_20.p","rb"))

In [140]:
real = pickle.load(open("saves/save_preds.p","rb"))

In [141]:
out = []
for x in prediction:
    out.append(x.replace("<|endoftext|>",""))
prediction = out

In [142]:
%%capture
pred = []
for x in prediction: 
    pred.append(tokenizer.decode(tokenizer.encode(x)[-1]))

In [143]:
counter = 0
for x in range(len(pred)):
    if pred[x] == real[x][1]:
        counter+=1
print(counter/len(pred))

0.3194


# Combining both

In [23]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [24]:
condition = "The Technology Report empowers or enlightens. ==== The Technology Report empowers or enlightens."

In [25]:
step1 = pickle.load(open("saves/save_preds_fine_1.p","rb"))
step1_corpus = lamT

In [26]:
%%capture
out= []
for x in range(len(step1)):
    final = step1[x][0]
    sentence = tokenizer.decode(lamT[x][:-1])
    out.append(sentence + final)

In [27]:
eos = split_eos(out)

In [ ]:
# full

In [39]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream/pytorch_model.bin"))
model = model.to("cuda")

In [40]:
%%capture
predictions = []
for x in eos: 
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_combined.p","wb"))

In [ ]:
# 6layer

In [28]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_final_6/pytorch_model.bin"))
model = model.to("cuda")

In [29]:
%%capture
predictions = []
count = 0
for x in eos: 
    count +=1
    if count%100 == 0: 
        print(count)
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_combined_6.p","wb"))

In [ ]:
# load 

In [30]:
prediction = pickle.load(open("saves/save_preds_combined_6.p","rb"))
real = pickle.load(open("saves/save_preds.p","rb"))

In [31]:
out = []
for x in prediction:
    out.append(x.replace("<|endoftext|>",""))
prediction = out

In [32]:
%%capture
pred = []
for x in prediction: 
    pred.append(tokenizer.decode(tokenizer.encode(x)[-1]))

In [33]:
counter = 0
for x in range(len(pred)):
    if pred[x] == real[x][1]:
        counter+=1
print(counter/len(pred))

0.3156


# What are the specific cases were there models diverge? (prediction,real)

In [40]:
predictions = pickle.load(open("saves/save_preds.p","rb"))

In [91]:
real = []
for x in predictions:
    real.append(x[1])

In [41]:
predictions2 = pickle.load(open("saves/save_preds_fine_1.p","rb"))

In [42]:
predictions3 = pickle.load(open("saves/save_preds_fine_20.p","rb"))

In [43]:
predictions4 = pickle.load(open("saves/save_preds_trans_no_trans_20.p","rb"))

In [55]:
predictions5 = pickle.load(open("saves/save_preds_trans.p","rb"))
tokens = []
for x in predictions5:
    tokens.append(tokenizer.decode(tokenizer.encode(x)[-2:-1]))
predictions5 = tokens

In [56]:
predictions6 = pickle.load(open("saves/save_preds_combined.p","rb"))
tokens = []
for x in predictions6:
    tokens.append(tokenizer.decode(tokenizer.encode(x)[-2:-1]))
predictions6 = tokens

In [132]:
correct = {}
correct["base"] = []
correct["fine1"] = []
correct["fine20"] = []
correct["transgen"] = []
correct["trans"] = []
correct["combo"] = []
for x in range (5000):
    if real[x] == predictions[x][0]:
        correct["base"].append(x)
    if real[x] == predictions2[x][0]:
        correct["fine1"].append(x)
    if real[x] == predictions3[x][0]:
        correct["fine20"].append(x) 
    if real[x] == predictions4[x][0]:
        correct["transgen"].append(x)  
    if real[x] == predictions5[x]:
        correct["trans"].append(x)
    if real[x] == predictions6[x]:
        correct["combo"].append(x)

In [ ]:
for x in correct["base"]: 
    if x not in correct["fine1"]:
        print(x)

In [142]:
predictions[2398]

(' sound', ' sound')

In [143]:
predictions2[2398]

(' that', ' sound')

In [144]:
tokenizer.decode(lamT[2398])

"there was only one person she knew who might be able to help grant. her hands were shaking when she dialed keith's number. he was the only defense lawyer she knew, and she didn't know where else to turn. `` isabelle, '' he greeted her. she could hear the smile in his voice, and it helped calm her nerves a bit to know he was still safe and sound"